In [1]:
import sys
sys.path.insert(1, '../../Src')
from utils.preprocessing import *



import matplotlib.pyplot as plt
import os, sys, gc, time, warnings, pickle, psutil, random
from math import ceil
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import numpy as np
import pandas as pd
import os

warnings.filterwarnings('ignore')


PATH_DATASET = '../../dataset/'
PATH_RESULTS = '../../results/Demanda/'

FILL = False
TARGET = 'Demanda'

SHIFT = 1

replace_nan = False
THRESHOLD_NAN = 0.6

%load_ext autoreload
%autoreload 2


In [2]:
df_test   = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','df_test.pkl'))
df_test.head()

,Z_MODELO,Z_PUNTO_VENTA,Z_GAMA,Z_WEEK,Z_WEEK_DATE,Demanda,Z_MARCA,Z_DEPARTAMENTO,date_block_num,item_id
0,MOD_1,PVENT_1,GAM_1,SEMANA_51,2022-05-02,0,MAR_2,DEP_1,50,MOD_1|PVENT_1|GAM_1
1,MOD_1,PVENT_1,GAM_1,SEMANA_52,2022-05-09,0,MAR_2,DEP_1,51,MOD_1|PVENT_1|GAM_1
2,MOD_1,PVENT_1,GAM_1,SEMANA_53,2022-05-16,0,MAR_2,DEP_1,52,MOD_1|PVENT_1|GAM_1
3,MOD_1,PVENT_1,GAM_1,SEMANA_54,2022-05-23,0,MAR_2,DEP_1,53,MOD_1|PVENT_1|GAM_1
4,MOD_1,PVENT_1,GAM_1,SEMANA_55,2022-05-30,0,MAR_2,DEP_1,54,MOD_1|PVENT_1|GAM_1


In [3]:
df_test.shape

(471730, 10)

In [4]:
print('Reading dataset')
df_train  = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','df_train.pkl'))
df_test   = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','df_test.pkl'))

df_train["item_id"] = df_train["Z_MODELO"].astype(str) +"|"+ df_train["Z_PUNTO_VENTA"].astype(str) +"|"+ df_train["Z_GAMA"].astype(str) 
df_test["item_id"] = df_test["Z_MODELO"].astype(str) +"|"+ df_test["Z_PUNTO_VENTA"].astype(str) +"|"+ df_test["Z_GAMA"].astype(str) 


df_release = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','grid_part_1.pkl'))
print('df_release size ',df_release.shape)

df_test.merge(df_release[['date_block_num','item_id','release']],on=['date_block_num','item_id'],how='left')
df_train.merge(df_release[['date_block_num','item_id','release']],on=['date_block_num','item_id'],how='left')
del df_release
gc.collect()

print('df_train size :',df_train.shape)
print('df_test  size :',df_test.shape)

Reading dataset
df_release size  (2830380, 11)
df_train size : (2358650, 10)
df_test  size : (471730, 10)


In [5]:
df_train.head()

,Z_MODELO,Z_PUNTO_VENTA,Z_GAMA,Z_WEEK,Z_WEEK_DATE,Demanda,Z_MARCA,Z_DEPARTAMENTO,date_block_num,item_id
0,MOD_1,PVENT_1,GAM_1,SEMANA_01,2021-05-17,0,MAR_2,DEP_1,0,MOD_1|PVENT_1|GAM_1
1,MOD_1,PVENT_1,GAM_1,SEMANA_02,2021-05-24,0,MAR_2,DEP_1,1,MOD_1|PVENT_1|GAM_1
2,MOD_1,PVENT_1,GAM_1,SEMANA_03,2021-05-31,0,MAR_2,DEP_1,2,MOD_1|PVENT_1|GAM_1
3,MOD_1,PVENT_1,GAM_1,SEMANA_04,2021-06-07,0,MAR_2,DEP_1,3,MOD_1|PVENT_1|GAM_1
4,MOD_1,PVENT_1,GAM_1,SEMANA_05,2021-06-14,0,MAR_2,DEP_1,4,MOD_1|PVENT_1|GAM_1


# Add features 

In [6]:

df_train.replace([np.inf, -np.inf, np.nan],0,inplace=True)
df_test.replace([np.inf, -np.inf, np.nan],0,inplace=True)

gc.collect()

0

In [7]:
df_test[TARGET] = -1

In [8]:
df_test.head()

,Z_MODELO,Z_PUNTO_VENTA,Z_GAMA,Z_WEEK,Z_WEEK_DATE,Demanda,Z_MARCA,Z_DEPARTAMENTO,date_block_num,item_id
0,MOD_1,PVENT_1,GAM_1,SEMANA_51,2022-05-02,-1,MAR_2,DEP_1,50,MOD_1|PVENT_1|GAM_1
1,MOD_1,PVENT_1,GAM_1,SEMANA_52,2022-05-09,-1,MAR_2,DEP_1,51,MOD_1|PVENT_1|GAM_1
2,MOD_1,PVENT_1,GAM_1,SEMANA_53,2022-05-16,-1,MAR_2,DEP_1,52,MOD_1|PVENT_1|GAM_1
3,MOD_1,PVENT_1,GAM_1,SEMANA_54,2022-05-23,-1,MAR_2,DEP_1,53,MOD_1|PVENT_1|GAM_1
4,MOD_1,PVENT_1,GAM_1,SEMANA_55,2022-05-30,-1,MAR_2,DEP_1,54,MOD_1|PVENT_1|GAM_1


In [9]:
############## dates ##################################

df_fe_dates = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','grid_part_2.pkl'))

print('df_fe_dates size :',df_fe_dates.shape)

df_train = df_train.merge(df_fe_dates,on=['item_id','date_block_num'],how='left')
df_test  = df_test.merge(df_fe_dates,on=['item_id','date_block_num'],how='left')

if replace_nan:
    print('replace_nan',replace_nan)
    df_train.replace([np.inf, -np.inf, np.nan],0,inplace=True)
    df_test.replace([np.inf, -np.inf, np.nan],0,inplace=True)

del df_fe_dates
gc.collect()
print('df_train size:',df_train.shape)
print('df_test size :',df_test.shape)

df_fe_dates size : (2830380, 14)
df_train size: (2358650, 22)
df_test size : (471730, 22)


In [10]:
df_test.head()

,Z_MODELO,Z_PUNTO_VENTA,Z_GAMA,Z_WEEK,Z_WEEK_DATE,Demanda,Z_MARCA,Z_DEPARTAMENTO,date_block_num,item_id,...,day,day_of_week,day_of_year,is_year_start,is_quarter_start,is_month_start,is_month_end,tm_wm,tm_w_end,tm_m_end
0,MOD_1,PVENT_1,GAM_1,SEMANA_51,2022-05-02,-1,MAR_2,DEP_1,50,MOD_1|PVENT_1|GAM_1,...,2,0,122,False,False,False,False,1,0,0
1,MOD_1,PVENT_1,GAM_1,SEMANA_52,2022-05-09,-1,MAR_2,DEP_1,51,MOD_1|PVENT_1|GAM_1,...,9,0,129,False,False,False,False,2,0,0
2,MOD_1,PVENT_1,GAM_1,SEMANA_53,2022-05-16,-1,MAR_2,DEP_1,52,MOD_1|PVENT_1|GAM_1,...,16,0,136,False,False,False,False,3,0,1
3,MOD_1,PVENT_1,GAM_1,SEMANA_54,2022-05-23,-1,MAR_2,DEP_1,53,MOD_1|PVENT_1|GAM_1,...,23,0,143,False,False,False,False,4,0,1
4,MOD_1,PVENT_1,GAM_1,SEMANA_55,2022-05-30,-1,MAR_2,DEP_1,54,MOD_1|PVENT_1|GAM_1,...,30,0,150,False,False,False,False,5,0,1


In [11]:
df_fe_lags = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','lags_df_'+str(SHIFT)+'_completed.pkl'))


In [12]:
############## LAGS ##################################

df_fe_lags = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','lags_df_'+str(SHIFT)+'_completed.pkl'))
del df_fe_lags[TARGET]
del df_fe_lags['Z_WEEK']
del df_fe_lags['Z_WEEK_DATE']

print('df_fe_lags size :',df_fe_lags.shape)


df_train = df_train.merge(df_fe_lags,on=['item_id','date_block_num'],how='left')
df_test  = df_test.merge(df_fe_lags,on=['item_id','date_block_num'],how='left')

if replace_nan:
    print('replace_nan',replace_nan)
    df_train.replace([np.inf, -np.inf, np.nan],0,inplace=True)
    df_test.replace([np.inf, -np.inf, np.nan],0,inplace=True)

del df_fe_lags
gc.collect()
print('df_train size:',df_train.shape)
print('df_test size :',df_test.shape)


df_fe_lags size : (2830380, 1042)
df_train size: (2358650, 1062)
df_test size : (471730, 1062)


In [13]:
df_test.head()

,Z_MODELO,Z_PUNTO_VENTA,Z_GAMA,Z_WEEK,Z_WEEK_DATE,Demanda,Z_MARCA,Z_DEPARTAMENTO,date_block_num,item_id,...,shift_1_roll_2_std_Z_MARCA,shift_1_roll_2_mean_Z_MARCA,shift_1_roll_2_std_Z_GAMA,shift_1_roll_2_mean_Z_GAMA,shift_1_roll_2_std_Z_PUNTO_VENTA,shift_1_roll_2_mean_Z_PUNTO_VENTA,shift_1_roll_2_std_Z_MODELO,shift_1_roll_2_mean_Z_MODELO,shift_1_roll_2_std_item_id,shift_1_roll_2_mean_item_id
0,MOD_1,PVENT_1,GAM_1,SEMANA_51,2022-05-02,-1,MAR_2,DEP_1,50,MOD_1|PVENT_1|GAM_1,...,139.5,357.5,139.5,357.5,139.5,357.5,139.5,357.5,139.5,357.5
1,MOD_1,PVENT_1,GAM_1,SEMANA_52,2022-05-09,-1,MAR_2,DEP_1,51,MOD_1|PVENT_1|GAM_1,...,109.0,109.0,109.0,109.0,109.0,109.0,109.0,109.0,109.0,109.0
2,MOD_1,PVENT_1,GAM_1,SEMANA_53,2022-05-16,-1,MAR_2,DEP_1,52,MOD_1|PVENT_1|GAM_1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,MOD_1,PVENT_1,GAM_1,SEMANA_54,2022-05-23,-1,MAR_2,DEP_1,53,MOD_1|PVENT_1|GAM_1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,MOD_1,PVENT_1,GAM_1,SEMANA_55,2022-05-30,-1,MAR_2,DEP_1,54,MOD_1|PVENT_1|GAM_1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
for column in df_test.columns:
    if 'DATE' in column:
        print(column)

Z_WEEK_DATE


In [15]:
from math import ceil

def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """

    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))

In [16]:
df_train['week_of_month'] = df_train['Z_WEEK_DATE'].apply(week_of_month)
df_test['week_of_month']  = df_test['Z_WEEK_DATE'].apply(week_of_month)


In [17]:
############## Encodings ##################################

df_fe_encodings = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','mean_encoding_df.pkl'))

del df_fe_encodings['Z_WEEK_DATE']

print('df_fe_encodings size :',df_fe_encodings.shape)

df_train = df_train.merge(df_fe_encodings,on=['item_id','date_block_num'],how='left')
df_test  = df_test.merge(df_fe_encodings,on=['item_id','date_block_num'],how='left')


if replace_nan:
    print('replace_nan',replace_nan)
    df_train.replace([np.inf, -np.inf, np.nan],0,inplace=True)
    df_test.replace([np.inf, -np.inf, np.nan],0,inplace=True)


del df_fe_encodings
gc.collect()
#df_train = reduce_mem_usage(df_train)
#df_test  = reduce_mem_usage(df_test)
gc.collect()
print('df_train size             :',df_train.shape)
print('df_test size :',df_test.shape)


df_fe_encodings size : (2830380, 50)
df_train size             : (2358650, 1111)
df_test size : (471730, 1111)


In [18]:
df_test.head()

,Z_MODELO,Z_PUNTO_VENTA,Z_GAMA,Z_WEEK,Z_WEEK_DATE,Demanda,Z_MARCA,Z_DEPARTAMENTO,date_block_num,item_id,...,enc_Z_GAMA_Z_MARCA_max,enc_Z_GAMA_Z_DEPARTAMENTO_mean,enc_Z_GAMA_Z_DEPARTAMENTO_std,enc_Z_GAMA_Z_DEPARTAMENTO_max,enc_Z_MARCA_Z_DEPARTAMENTO_mean,enc_Z_MARCA_Z_DEPARTAMENTO_std,enc_Z_MARCA_Z_DEPARTAMENTO_max,enc_Z_MODELO_Z_PUNTO_VENTA_Z_GAMA_mean,enc_Z_MODELO_Z_PUNTO_VENTA_Z_GAMA_std,enc_Z_MODELO_Z_PUNTO_VENTA_Z_GAMA_max
0,MOD_1,PVENT_1,GAM_1,SEMANA_51,2022-05-02,-1,MAR_2,DEP_1,50,MOD_1|PVENT_1|GAM_1,...,762.0,1.218529,9.906863,1032.0,1.325033,12.110056,1451.0,56.68,124.736694,497.0
1,MOD_1,PVENT_1,GAM_1,SEMANA_52,2022-05-09,-1,MAR_2,DEP_1,51,MOD_1|PVENT_1|GAM_1,...,762.0,1.218529,9.906863,1032.0,1.325033,12.110056,1451.0,56.68,124.736694,497.0
2,MOD_1,PVENT_1,GAM_1,SEMANA_53,2022-05-16,-1,MAR_2,DEP_1,52,MOD_1|PVENT_1|GAM_1,...,762.0,1.218529,9.906863,1032.0,1.325033,12.110056,1451.0,56.68,124.736694,497.0
3,MOD_1,PVENT_1,GAM_1,SEMANA_54,2022-05-23,-1,MAR_2,DEP_1,53,MOD_1|PVENT_1|GAM_1,...,762.0,1.218529,9.906863,1032.0,1.325033,12.110056,1451.0,56.68,124.736694,497.0
4,MOD_1,PVENT_1,GAM_1,SEMANA_55,2022-05-30,-1,MAR_2,DEP_1,54,MOD_1|PVENT_1|GAM_1,...,762.0,1.218529,9.906863,1032.0,1.325033,12.110056,1451.0,56.68,124.736694,497.0


# Save Data

In [19]:
########################### Save part 1
#################################################################################
print('Save Dataset ..')

print('saving .. ',os.path.join(PATH_RESULTS,'dataset','df_train_fe_FILL_'+str(FILL)+'_SHIFT_'+str(SHIFT)+'.pkl'))
df_train.to_pickle(os.path.join(PATH_RESULTS,'dataset','df_train_fe_FILL_'+str(FILL)+'_SHIFT_'+str(SHIFT)+'.pkl'))

print('df_train Size:', df_train.shape)

print('saving .. ',os.path.join(PATH_RESULTS,'dataset','df_test_fe_FILL_'+str(FILL)+'_SHIFT_'+str(SHIFT)+'.pkl'))

df_test.to_pickle(os.path.join(PATH_RESULTS,'dataset','df_test_fe_FILL_'+str(FILL)+'_SHIFT_'+str(SHIFT)+'.pkl'))

print('df_test Size:', df_test.shape)

Save Dataset ..
saving ..  ../../results/Demanda/dataset/df_train_fe_FILL_False_SHIFT_1.pkl
df_train Size: (2358650, 1111)
saving ..  ../../results/Demanda/dataset/df_test_fe_FILL_False_SHIFT_1.pkl
df_test Size: (471730, 1111)
